In [2]:
#some ideas from Fayem Hathak
import numpy as np
import scipy.io.wavfile 
import math
import sys
from scipy.fftpack import dct

In [3]:
first_file = 'digits/1a.wav'
(sample_rate,signal) = scipy.io.wavfile.read(first_file)
signal

array([2, 2, 0, ..., 2, 1, 2], dtype=int16)

In [4]:
# take 256 points FFT
window_size = 256
# shift = 20
#pre_emphasis

# pre-Emphasis
alpha = 0.97 #preemphais coeff
emphasized_signal = np.append(signal[0], signal[1:] - alpha * signal[:-1])
# hamming = np.hamming(window_size)# hamming window 
# window_start = np.arange(0,new_signal.shape[0]-window_size,shift) # position of the start of the window
# # windowing
# frames = np.zeros([window_start.shape[-1],window_size],dtype = complex)
# # frame
# for i in np.arange(0,window_start.shape[-1]):
#     start = window_start[i]
#     X = np.fft.fft(new_signal[start:(start+window_size)]*hamming)
#     frames[i,:] = X
frame_size = 0.02
frame_stride = 0.01
emphasized_signal

array([ 2.  ,  0.06, -1.94, ...,  1.03, -0.94,  1.03])

In [5]:
alpha = 0.97 #preemphais coeff
new_signal = np.zeros(len(signal))
for i in range(len(signal)):
    if i is 0:
        new_signal[i] = signal[0]
    else:
        new_signal[i] = signal[i]- alpha*signal[i-1]
new_signal.shape[0]

12083

In [6]:
frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)# length of the signals
frame_length = int(round(frame_length)) # round the frame_length
frame_step = int(round(frame_step)) # get the step of the frame
num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
pad_signal_length = num_frames * frame_step + frame_length
z = np.zeros((pad_signal_length - signal_length))
pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).transpose()
frames = pad_signal[indices.astype(np.int32, copy=False)]
# np.shape(frames)
indices[0]
# pad_signal.shape[0]
# frame_length
# np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1))
# pad_signal
indices
frames

array([[ 2.  ,  0.06, -1.94, ...,  0.  ,  0.  , -2.  ],
       [-0.85,  1.12, -1.85, ..., -0.03,  0.97, -1.  ],
       [ 1.94, -1.  ,  0.97, ...,  1.88, -5.06, -5.21],
       ...,
       [-0.88,  1.09,  2.12, ..., -0.91,  2.06, -2.88],
       [ 0.09,  1.09, -0.88, ..., -0.21,  0.79, -3.18],
       [ 1.03,  1.06, -0.91, ...,  1.73, -1.21,  0.76]])

In [7]:
frame_length
indices = np.zeros(shape=(num_frames,frame_length))
prevx = 0
x = 320
for i in range(num_frames):
    indices[i] = np.arange(prevx,x)
    prevx += frame_step
    x+=frame_step
indices = indices.astype(np.int32, copy=False)
# len(signal)
# np.shape(frames)
# pad_signal[indices]
frames = pad_signal[indices]
frames

array([[ 2.  ,  0.06, -1.94, ...,  0.  ,  0.  , -2.  ],
       [-0.85,  1.12, -1.85, ..., -0.03,  0.97, -1.  ],
       [ 1.94, -1.  ,  0.97, ...,  1.88, -5.06, -5.21],
       ...,
       [-0.88,  1.09,  2.12, ..., -0.91,  2.06, -2.88],
       [ 0.09,  1.09, -0.88, ..., -0.21,  0.79, -3.18],
       [ 1.03,  1.06, -0.91, ...,  1.73, -1.21,  0.76]])

In [8]:
# extract the 12 features
frames *= np.hamming(frame_length)
mag_frames = np.absolute(np.fft.rfft(frames, window_size))
pow_frames = ((1.0 / window_size) * ((mag_frames) ** 2))
# for i  in wi
# we did not have duplicates
high_mel_freq = 1125 * np.log(1 + (sample_rate/2) / 700)
low_mel_freq = 0
num_filter = 40 # number of filters
mel_points = np.linspace(low_mel_freq, high_mel_freq, num_filter + 2)
# in hertz
hz_points = 700 * (math.e**(mel_points / 1127) - 1)
bin = np.floor((window_size + 1) * hz_points / sample_rate)
# filter banks
fbank = np.zeros((num_filter, int(np.floor(window_size/2 + 1))))
for m in range(1, num_filter+1):
    f_m_left = int(bin[m - 1])    # left
    f_m = int(bin[m])             # center
    f_m_right = int(bin[m + 1])   # right
    for k in range(f_m_left, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1]) #implement the filtering
    for k in range(f_m, f_m_right):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m]) #implement the filtering
# dot product between each bank with the power sum of the frame       
filter_banks = np.dot(pow_frames, fbank.transpose()) # find the forier  
filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * np.log(filter_banks) # take the logs
# filter_banks
num_ceps = 12 #number of ceps coefficients
filter_banks
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # keep ceps from 2 to 13

In [9]:
np.shape(mfcc)

(74, 12)

In [10]:
np.dot(mfcc[1,:],mfcc[1,:].T)

248974.49319895642

In [11]:
len(mfcc)

74

In [14]:
pow_frames.shape[-1]

129

In [15]:
frames *= np.hamming(frame_length)

In [16]:
np.shape(frames)

(74, 320)

In [18]:
np.shape(mag_frames)

(74, 129)

In [19]:
np.shape(fbank)

(40, 129)

In [20]:
np.shape(filter_banks)

(74, 40)

In [24]:
np.shape(np.fft.rfft(frames,window_size))

(74, 129)

array([[  5.66694135 +0.        j,  -7.83143875 -3.7426522 j,
          8.69090335 +0.79335612j, ...,   1.92750555 -3.82024472j,
         -5.67531606 +1.6999376 j,   5.1914659  +0.        j],
       [  1.38753902 +0.        j,   2.86398499 +0.64657267j,
          0.65839389 +4.94251052j, ...,  -3.98871772 -9.30243785j,
         -0.4313693  -2.19197974j,   9.10835912 +0.        j],
       [ -2.06131498 +0.        j,   3.8881209  -2.80774911j,
         -2.34235549 +3.25983222j, ...,  -3.81877438 -6.90944923j,
          2.17563972 +2.66170863j,   0.54422902 +0.        j],
       ...,
       [ 12.98638296 +0.        j,  -5.83813567+12.45930684j,
         -2.89162553 -7.24253055j, ...,  10.61843311 -2.50227506j,
          3.08307523+15.05458213j, -12.57311373 +0.        j],
       [ 11.24599926 +0.        j,  -5.40919731 +1.58579549j,
          4.81055337 +1.93850975j, ...,  -0.90897635-18.59855511j,
         11.83469052 +5.46820128j,  -7.95439268 +0.        j],
       [-32.59557305 +0.    